In [1]:
import os
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import trompy as tp
import pandas as pd
from scipy import stats

In [2]:
def get_number_cells (s2p_folder):
    iscell = np.load(os.path.join(s2p_folder, 'iscell.npy'))
    return sum(iscell[:,0])
    

In [3]:
def get_frames(events_file,s2p_length):
    df = pd.read_csv(events_file)

    pump_frames = list(df[df['Item1'] == 'pump_on']['Item2.Item2'])

    pump_frames=[frame for frame in pump_frames if frame < s2p_length-100] #removes event if too close to the end

    licks_frames = list (df[df['Item1']== 'Lick']['Item2.Item2'])
    
    return pump_frames, licks_frames

In [4]:
def filter_cell(x):

    t = np.arange(0,len(x)/10,0.1)
    filt = signal.butter(4, 1, 'low', fs=10, output='sos')
    filtered = signal.sosfilt(filt, x)
    
    return filtered
    

In [17]:
def get_activated_cells (s2p_folder,events_file):
    raw_F = np.load(os.path.join(s2p_folder, 'F.npy'))
    neu_F = np.load(os.path.join(s2p_folder, 'Fneu.npy'))
    iscell = np.load(os.path.join(s2p_folder, 'iscell.npy'))
#     stats=np.load(os.path.join(s2p_folder, 'stat.npy'), allow_pickle=True)
    
    s2p_length=len(raw_F[0,:])
    print(s2p_length)
    
    pump_frames, licks_frames= get_frames(events_file,s2p_length)
    
    cell_idx = [idx for idx,vals in enumerate(iscell) if vals[0]==1]
    
    activated_cells=0

    for i, cell in enumerate(cell_idx):
        x = raw_F[cell, :] - 0.7*(neu_F[cell, :])
        x = raw_F[cell, :]
        x = (x - np.mean(x))/np.std(x)
        x = filter_cell(x)
        
        
        snips = []
        pre=[]
        post=[]
        
        for p in pump_frames:
            snips.append(x[p-50:p+100])
            pre.append(np.mean(x[p-50:p]))
            post.append(np.mean(x[p:p+50]))
        result= stats.ttest_ind(pre, post)
        
        print(result[1])
        
# paired t-test
#hit trials


In [18]:
animal="VGP302"
day="031"
date="20221110"
s2p_folder = "C:\\Users\\mko102\\UiT Office 365\\O365-Pipette 2 - Dokumenter\\People\\Mette\\GitHub\\2P\\Processed\\VGP_procd_data_22-12-05\\sub-{}\\ses-{}".format(animal, day)
events_file = "C:\\Users\\mko102\\UiT Office 365\\O365-Pipette 2 - Dokumenter\\People\\Mette\\GitHub\\2P\\Processed\\behav\\sub-{0}\\ses-{1}-{2}\\sub-{0}_ses-{1}_events.csv".format(animal, day, date)


In [19]:
get_number_cells(s2p_folder)

76.0

In [20]:
get_activated_cells(s2p_folder,events_file)

11680
0.8813977297981109
0.7859575681040613
0.42264044844512494
0.6610098666126161
0.9895217348615957
0.9925517142810266
0.972676691964709
0.9058220660429561
0.9645292071451894
0.785549245032089
0.9454973820790022
0.47804540848874544
0.6816889483793537
0.9305406642309428
0.6896683089311197
0.8825977624564799
0.9039800163949154
0.534001166486227
0.6923130428706274
0.9898974149259889
0.7895930319721837
0.6652541294566572
0.7873151522505004
0.5913526312515818
0.9962963731025647
0.8414995052903755
0.8363991261553017
0.9972567927510362
0.6442464792585956
0.8289671716503979
0.9129273153475667
0.9800552588397354
0.40796359591219433
0.9922055056090823
0.7476851238129197
0.6041000228176209
0.6216698066870932
0.7898311577186617
0.4592069697065535
0.8630167308487253
0.2507137683508466
0.41820152631918583
0.2459941325695157
0.9089866005157152
0.7220189545539685
0.2576511705268936
0.583919058053048
0.8869294713114851
0.7741191234523879
0.8674389486109828
0.5768227056637267
0.1761986514655879
0.7272